# Logic Simulation

In [14]:
# Importing Libraries
import numpy as np
import networkx as nx
from collections import deque

We start with a function that reads the netlist and returns a list of tuples (*nodes*) of the joint edges and dictionary (*gates) which classifies the nodes as primary input or a gate type.

`add_gate_type()` nested function is used to classify nodes as gates.

In [15]:
def readNet(file):
    try: 
        with open(file, "r") as filehandle:
            data = filehandle.read().split("\n")
        # Raise exception if file not found
    except Exception as ex:
        print(ex)
        return

    # Classifying nodes as PI, gates
    def add_gate_type(gate_name, gate_type, gate_dict):
        if gate_name not in gate_dict.keys():
            gate_dict[gate_name] = gate_type
        else:
            if gate_dict[gate_name] == 'PI':
                gate_dict[gate_name] = gate_type

    nodes = []
    gates = {}
    for line in data:
        # Skip for blank lines
        if line == '':
            continue
        i = line.split()
        if i[1] == 'buf':
            nodes.append((i[2], i[3]))
            add_gate_type(i[2], 'PI', gates)    
            add_gate_type(i[3], 'buf', gates)    
            continue
        if i[1] == 'inv':
            nodes.append((i[2], i[3]))
            add_gate_type(i[2], 'PI', gates)    
            add_gate_type(i[3], 'inv', gates)    
        else:
            nodes.append((i[2], i[4]))
            nodes.append((i[3], i[4]))
            add_gate_type(i[2], 'PI', gates)
            add_gate_type(i[3], 'PI', gates)
            add_gate_type(i[4], i[1], gates)
            
    return nodes, gates

We then define a function to perform bit operations from the input bits of the node predecessors. </br>
We pass gate_dict as an argument to identify the output gate type.

In [16]:
def gate_op(node, bit1, bit2, gate_dict):
    gate_type = gate_dict[node]
    if gate_type == 'inv':
        return int(not(bit1))
    elif gate_type == 'and2':
        return int((bit1 and bit2))
    elif gate_type == 'or2':
        return int((bit1 or bit2))
    elif gate_type == 'nor2':
        return int(not(bit1 or bit2))
    elif gate_type == 'nand2':
        return int(not(bit1 and bit2))
    elif gate_type == 'xor2':
        return int(bit1^bit2)
    elif gate_type == 'xnor2':
        return int(not(bit1^bit2))
    elif gate_type == 'buf':
        return int(bit1)

In [17]:
# Defining a function to perform bitwise XOR and XNOR
def xor(x, y):
    return int((x and not y) and (not x and y))

def xnor(x, y):
    return int(not((x and not y) and (not x and y)))    

`topo_simulate_gate()` and `event_simulate_gate()` are functions that takes the netlist and input files as arguments. If either file paths are wrong, exceptions are raised.

Upon evaluating the logic circuit, a new file *out_file* is created and output vectors are stored in alphabetical order of net-names.

We read the netlist using `readNet()` which returns the edges and gate-type dictionaries. </br>
We create an acyclic graph using the imported `networkx` library. If cycles are found in the graph, or simply the circuit has combinational loops, an exception is raised.

## Topologically ordered evaluation

**Algorithm:**
After a Digraph is created, we use `topological_sort()` to arrange the nets in topological order.

We then read bits from the input files and apply logic operations in accordance with our topological order.

output_dict is a dictionary holding output vectors as lists. For non-PI type nodes, we access this dictionary to find its predecessor node bits and apply logic operations to find the output bit.

In [18]:
def topo_simulate_gate(net_file, inp_file, out_file):
    nodes, gates = readNet(net_file)
    graph = nx.DiGraph()
    graph.add_edges_from(nodes)

    # Raise exception for combinational loops
    if(nx.is_directed_acyclic_graph(graph) == False):
        raise Exception("Error in netlist, combinational loops found")
    
    # Topologically sort
    nl = list(nx.topological_sort(graph))
        
    output_dict = {}
    gate_inputs = [k for k,v in gates.items() if v == 'PI']
    # Sort PI type nodes alphabetically
    gate_inputs.sort(key=lambda item: (len(item), item))
    
    # Assign an dictionary to hold output vectors
    for i in range(len(nl)):
        output_dict[nl[i]] = []
    
    try: 
        with open(inp_file, "r") as filehandle:
            data = filehandle.read().split("\n")
        # Raise exception if file not found
    except Exception as ex:
        print(ex)
        exit()    

    for line in data[1:]:
        # Skip blank lines
        if line == '':
            continue
        i = line.split()
        # Read input file for PI nodes
        for j in range(len(gate_inputs)):
            output_dict[gate_inputs[j]].append((int(i[j])))
        
        # For non-PI nodes, apply logic operations to predecessors
        for j in range(len(gate_inputs), len(nl), 1):
            in1 = output_dict[list(graph.predecessors(nl[j]))[0]][-1]
            if gates[nl[j]] == 'inv' or gates[nl[j]] == 'buf':
                in2 = -1
            else:
                in2 = output_dict[list(graph.predecessors(nl[j]))[1]][-1]
            output_dict[nl[j]].append(gate_op(nl[j], in1, in2, gates))
    
    # Sort all gates alphabetically            
    ordered_gates = [k for k,v in output_dict.items()]
    ordered_gates.sort(key=lambda item: (len(item), item))
    
    # Write output to output file
    with open(out_file, 'w') as f:
        print(*ordered_gates, file=f)
        for i in range(len(output_dict[ordered_gates[0]])):
            bits = []
            for node in ordered_gates:
                bits.append(output_dict[node][i])
            print(*bits, file = f)

In [6]:
%timeit topo_simulate_gate('benchmarks/c17.net', 'benchmarks/c17.inputs', 'what_c17.outputs')

[Errno 2] No such file or directory: 'benchmarks/c17.net'


TypeError: cannot unpack non-iterable NoneType object

## Event-driven evaluation

**Algorithm:**
For the first output vector, we start with assigning x to the state table. *waitlist* is a queue of nodes. </br>
We first queue all the PI nodes to our waitlist, and start dequeing.

For PI nodes, we simply read the bit from the first line of the input file and dequeue it and append it to the list followed by queing all its succesor nodes.

*gate_bits* is a dictionary which holds lists for each node.

For non-PI nodes, we first check if its predecessors bits are defined or simply put, the list holding outputs is empty for that node or contains `None`. If this is the case, we simply dequeue the node with queing anything.
Else, we access its predecessor bits and apply gate operations to find the output bit and append it to the list followed by dequeing the nodes and queing all the successor nodes.

`bit_deque()` function returns the bit to be appended.</br> 
For PI nodes, the bit is returned. </br>
For non-PI nodes, we apply logic operations on the predecessor bits (if defined) to find the bit to append to the  node list.

We follow this process until the queue is empty (when the state table is defined for all the nodes). We then extract the latest bit for each node from its corresponding list as that is its true value.

After the state table is filled, we make a new queue *q*. We queue to q only if the input bit corresponding to that node is changed. If so, we dequeue the node followed by adding all its successor nodes and follow the same process for this node. 

`solve_queue()` is a recursive function which performs above algorithm until queue is emptied.

In [19]:
def event_simulate_gate(net_file, inp_file, out_file):
    outputFile = open(out_file, "w")
    edges, gate_type = readNet(net_file)
    event_graph = nx.DiGraph()
    event_graph.add_edges_from(edges)
    nx.set_node_attributes(event_graph, gate_type, name='gateType')

    # Raise exception for combinational loops
    if(nx.is_directed_acyclic_graph(event_graph) == False):
        raise Exception("Error in netlist, combinational loops found")

    # list holding all PI nodes
    input_gate = [k for k,v in gate_type.items() if v == 'PI']
    # Sorting in alphabetical order
    input_gate.sort(key=lambda item: (len(item), item))
    all_nodes = [k for k,v in gate_type.items()]

    # Queue for nodes
    waitlist = deque()
    # queing all primary inputs
    for i in input_gate:
        waitlist.append(i)
    # Reading input file
    try: 
        with open(inp_file, "r") as filehandle:
            data = filehandle.read().split("\n")

    # Raise exception if file not found
    except Exception as ex:
        print(ex)
        exit()
    
    # Dictionary to hold outputs
    gate_bits = {}
    for i in range(len(all_nodes)):
        gate_bits[all_nodes[i]] = []

    def bit_dequeue(node_name, pi_bit=-1):
        # Check if node is PI or not
        if pi_bit != -1:
            return pi_bit
        else:
        # Check if predecessors bit is defined   
            for pred in list(event_graph.predecessors(node_name)):
                if gate_bits[pred] == []:
                    return
                if gate_bits[pred][-1] == None:
                    return  
            # Apply gate operations
            if gate_type[node_name] == 'inv':
                return int(not(gate_bits[list(event_graph.predecessors(node_name))[0]][-1]))
            elif gate_type[node_name] == 'and2':
                return int((gate_bits[list(event_graph.predecessors(node_name))[0]][-1] and gate_bits[list(event_graph.predecessors(node_name))[1]][-1]))
            elif gate_type[node_name] == 'or2':
                return int((gate_bits[list(event_graph.predecessors(node_name))[0]][-1] or gate_bits[list(event_graph.predecessors(node_name))[1]][-1]))
            elif gate_type[node_name] == 'nand2':
                return int(not((gate_bits[list(event_graph.predecessors(node_name))[0]][-1] and gate_bits[list(event_graph.predecessors(node_name))[1]][-1])))
            elif gate_type[node_name] == 'nor2':
                return int(not((gate_bits[list(event_graph.predecessors(node_name))[0]][-1] or gate_bits[list(event_graph.predecessors(node_name))[1]][-1])))
            elif gate_type[node_name] == 'xor2':
                return xor(gate_bits[list(event_graph.predecessors(node_name))[0]][-1], gate_bits[list(event_graph.predecessors(node_name))[1]][-1])
            elif gate_type[node_name] == 'xnor2':
                return xnor(gate_bits[list(event_graph.predecessors(node_name))[0]][-1], gate_bits[list(event_graph.predecessors(node_name))[1]][-1])
            elif gate_type[node_name] == 'buf':
                return int(gate_bits[list(event_graph.predecessors(node_name))[0]][-1])

    # Reading first line from input file
    line = data[1]
    i = line.split()
    for j in range(len(i)):
        # Dequeing node
        node_dequeue = waitlist.popleft()
        gate_bits[node_dequeue].append(bit_dequeue(node_dequeue, pi_bit=int(i[j])))
        # Queing all successor nodes
        for succ in list(event_graph.successors(node_dequeue)):
            waitlist.append(succ)
            
    while(len(waitlist)>0):
        node_dequeue = waitlist.popleft()
        gate_bits[node_dequeue].append(bit_dequeue(node_dequeue))
        for succ in list(event_graph.successors(node_dequeue)):
            waitlist.append(succ)

    # Extracting the latest bit in the list, as that is the true value
    for i in all_nodes:
        gate_bits[i] = [gate_bits[i][-1]]

    ordered_gates = [k for k,v in gate_bits.items()]
    # Sorting alphabetically
    ordered_gates.sort(key=lambda item: (len(item), item))
    event_output = {}
    for i in ordered_gates:
        event_output[i] = [(gate_bits[i][0])]

    def solve_queue(node_id, new_bit, q, flag = 0):
        # Execute until queue is empty
        while(len(q) > 0 or flag == 0):
            flag = 1
            # Compare with old_bit
            if gate_bits[node_id][-1] == new_bit:
                gate_bits[node_id].append(new_bit)

            # If not same, queue all successor nodes and dequeue current node
            # by applying logic operations on predecessor nodes
            else:
                # queue current node
                q.append(node_id)
                gate_bits[node_id].append(new_bit)
                q.popleft()
                for succ in list(event_graph.successors(node_id)):
                    in1 = gate_bits[list(event_graph.predecessors(succ))[0]][-1]
                    if gate_type[succ] == 'inv' or gate_type[succ] == 'buf':
                        in2 = -1
                    else:
                        in2 = gate_bits[list(event_graph.predecessors(succ))[1]][-1]
                        output_new_bit = gate_op(succ, in1, in2, gate_type)
                        solve_queue(succ, output_new_bit, q)

    # Writing to output file      
    for j in ordered_gates:
        outputFile.write(str(j) + " ")
    outputFile.write('\n')
    for j in ordered_gates:
        outputFile.write(str(gate_bits[j][0]) + " ")
    outputFile.write("\n")
    for line in data[2:]:
        outputLine = ""
        if line == ''        :
            continue
        i = line.split()
        for j in range(len(input_gate)):
            # Queue for nodes with mismatching bits
            state_queue = deque()
            solve_queue(input_gate[j], int(i[j]), state_queue)
        for j in ordered_gates:
            gate_bits[j] = [gate_bits[j][-1]]
            outputLine += str(gate_bits[j][0]) + " "
        outputFile.write(outputLine + '\n')  
    outputFile.close() 


In [20]:
%timeit event_simulate_gate('benchmarks/c17.net', 'benchmarks/c17.inputs', 'c17_event.outputs')

[Errno 2] No such file or directory: 'benchmarks/c17.net'


TypeError: cannot unpack non-iterable NoneType object

## Comparing outputs

In [21]:
print('Using event based evaluation:')
%timeit event_simulate_gate('benchmarks/c17.net', 'benchmarks/c17.inputs', 'what_c17_event.outputs')
%timeit event_simulate_gate('benchmarks/c432.net', 'benchmarks/c432.inputs', 'what_c432_event.outputs')
%timeit event_simulate_gate('benchmarks/c8.net', 'benchmarks/c8.inputs', 'what_c8_event.outputs')
%timeit event_simulate_gate('benchmarks/parity.net', 'benchmarks/parity.inputs', 'what_parity_event.outputs')

print('Using topological sorting:')
%timeit topo_simulate_gate('benchmarks/c17.net', 'benchmarks/c17.inputs', 'what_c17_event.outputs')
%timeit topo_simulate_gate('benchmarks/c432.net', 'benchmarks/c432.inputs', 'what_c432_event.outputs')
%timeit topo_simulate_gate('benchmarks/c8.net', 'benchmarks/c8.inputs', 'what_c8_event.outputs')
%timeit topo_simulate_gate('benchmarks/parity.net', 'benchmarks/parity.inputs', 'what_parity_event.outputs')

Using event based evaluation:
[Errno 2] No such file or directory: 'benchmarks/c17.net'


TypeError: cannot unpack non-iterable NoneType object

Upon running the functions for topological sorting as well as event-driven evaluation for different netlists, we obtain the following results:
- For the c17 netlist inputs, both the methods take a similar amount of time to run. 
- For the c432 netlist inputs, the event-driven method takes a much larger amount of time - more than 10 times that taken by the topological sorting method.

We infer that the time taken by the method depends on the size of the input dataset. c432 is a large dataset with multiple input values changing simultaneously from each state to the next, and there is a large number of states as well. Hence, evaluating outputs based on an event-driven method takes a large amount of time as it has to check for many values changing. On the other hand, the c17 dataset is small and there are fewer input values changing from one state to the next, so the event-driven method and the topological sorting method take similar amount of time.

